Team members:
- Sophia Gabriela Martinez Albarran A01424430
- Eduardo Botello Casey A01659281
- Marcos Saade Romano A01784220

## Dataset description

The data is related with direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict if the client will subscribe a term deposit (variable y). It contains 6 integers variables and 11 categorical variables. It has 45211 instances of data, where we will be performing a cleaning and preprocessing before creating the models.

In [8]:
import sklearn 
from ucimlrepo import fetch_ucirepo 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

  
# https://archive.ics.uci.edu/dataset/222/bank+marketing  
# fetch dataset 
bank_marketing = fetch_ucirepo(id=222) 
  
# data (as pandas dataframes) 
X = bank_marketing.data.features 
y = bank_marketing.data.targets 
  
# Drop the column duration because in the page it says it isn't necessary for a predictive model
X = X.drop(columns=['duration'])

Duration was removed based on the dataset documentation

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          45211 non-null  int64 
 1   job          44923 non-null  object
 2   marital      45211 non-null  object
 3   education    43354 non-null  object
 4   default      45211 non-null  object
 5   balance      45211 non-null  int64 
 6   housing      45211 non-null  object
 7   loan         45211 non-null  object
 8   contact      32191 non-null  object
 9   day_of_week  45211 non-null  int64 
 10  month        45211 non-null  object
 11  campaign     45211 non-null  int64 
 12  pdays        45211 non-null  int64 
 13  previous     45211 non-null  int64 
 14  poutcome     8252 non-null   object
dtypes: int64(6), object(9)
memory usage: 5.2+ MB


In [10]:
X.describe()

,age,balance,day_of_week,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,63.000000,871.000000,275.000000


In [11]:
X.job = X.job.fillna('unknown')
X.education = X.education.fillna('unknown')
X.poutcome = X.poutcome.fillna('nonexistent')
X.contact = X.contact.fillna('unknown')


Missing values in categorical variables were replaced with unknown in the database. 

Rather than deleting data, these values were preserved as valid categories.

One-hot encoding will later treat them as distinct features.



In [5]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')
X_encoded = encoder.fit_transform(X.select_dtypes(include=['object']))
X_encoded_df = pd.DataFrame(
    X_encoded.toarray(), 
    columns=encoder.get_feature_names_out(X.select_dtypes(include=['object']).columns)
)


In [12]:
X_numerico = X.select_dtypes(exclude=['object'])
X_final = pd.concat([X_numerico.reset_index(drop=True), X_encoded_df.reset_index(drop=True)], axis=1)

# Logistic Regression Classifier 

Logistic Regression is widely used for binary classification problems and serves as an interpretable and effective baseline.

### Advantages:

- Interpretability: The model's coefficients indicate the direction and strength of each feature’s influence on the prediction.

- Efficiency: Training and inference are fast, even with large datasets.

- Probabilistic Output: It provides confidence scores for predictions.

- Built-in Regularization: It supports L2 regularization to reduce overfitting.

### Disadvantages 

- Assumes a linear relationship between the input features and the log-odds of the outcome.

- Sensitive to feature scaling, requiring standardized numeric features.

- Not suitable for nonlinear relationships unless combined with feature engineering.

In [ ]:

cat_col = X.select_dtypes(include='object').columns.tolist()
num_col = X.select_dtypes(exclude='object').columns.tolist()


categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, cat_col),
        ('num', numerical_transformer, num_col)
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])


To ensure proper and consistent preprocessing of both categorical and numerical variables, we created a structured pipeline using ColumnTransformer and Pipeline. 
This approach is modular, safe against data leakage, and compatible with cross-validation and model tuning.

It automatically identifies categorical columns (those with object dtype) and stores them in cat_col, it stores the remaining numerical columns in num_col.

Then:

- OneHotEncoder: Converts each categorical column into a set of binary columns (0 or 1), one per category. It uses handle_unknown='ignore' to safely handle categories that might appear in new data but were not seen during training.

- StandardScaler: Standardizes numerical features by removing the mean and scaling to unit variance.

We use it because Logistic Regression is sensitive to feature scale, so we need standardized inputs for numeric features, and categorical variables must be encoded into numbers for the model to process them.

Then we built a preprocessing object that applies categorical_transformer to the categorical columns, and numerical_transformer to the numeric columns.It also applies different transformations in parallel and ensures each set of columns is processed as needed. It also integrates smoothly into a pipeline.

In [ ]:

y = y['y'].map({'yes': 1, 'no': 0})

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

The target variable y originally consisted of categorical values 'yes' and 'no'. These were converted into binary numeric values 1 and 0 respectively using .map(). This transformation was required to make the data compatible with scikit-learn classifiers, which expect numeric labels. This also formalizes the task as a binary classification problem

Then the dataset was split into training (80%) and test (20%) subsets using train_test_split(). The stratify=y parameter was used to maintain the original proportion of classes in both sets, which is particularly important for imbalanced classification problems. Additionally, random_state=42 was used to ensure that the split is reproducible, making the results consistent across runs.

In [15]:
from sklearn.model_selection import GridSearchCV

param = {
    'classifier__C': [0.01, 0.1, 1, 10],
    'classifier__penalty': ['l2'],
    'classifier__solver': ['lbfgs']
}

grid_search = GridSearchCV(pipeline, param, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)

best_logisticreg_model = grid_search.best_estimator_

Hyperparameter tuning was performed using GridSearchCV with 5-fold cross-validation. A grid of candidate values was defined for C, the regularization strength of the Logistic Regression model. We fixed the penalty to L2 and used the lbfgs solver due to its efficiency with this penalty. The model was evaluated using the ROC AUC metric during cross-validation, which is appropriate for this imbalanced classification task. The best pipeline configuration was stored in best_logisticreg_model and used for final evaluation.

In [16]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

y_pred = best_logisticreg_model.predict(X_test)
y_proba = best_logisticreg_model.predict_proba(X_test)[:, 1]

print("Mejores parámetros:", grid_search.best_params_)
print("Reporte de Clasificación:\n", classification_report(y_test, y_pred))
print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))


Mejores parámetros: {'classifier__C': 0.1, 'classifier__penalty': 'l2', 'classifier__solver': 'lbfgs'}
Reporte de Clasificación:
               precision    recall  f1-score   support

           0       0.90      0.99      0.94      7985
           1       0.67      0.17      0.27      1058

    accuracy                           0.89      9043
   macro avg       0.79      0.58      0.61      9043
weighted avg       0.87      0.89      0.86      9043

Matriz de Confusión:
 [[7897   88]
 [ 877  181]]
ROC AUC: 0.7725529791800079


The model achieves high precision (0.67) on the positive class (yes), meaning that when it predicts a client will subscribe, it is often correct.

However, it has low recall (0.17) on that same class, indicating that it misses a large portion of the actual positives.

This imbalance is common in datasets with skewed class distribution (most clients do not subscribe).

The high overall accuracy (0.89) is influenced by the model’s strong performance on the dominant class (no), but it may not be sufficient for applications where detecting potential subscribers is critical.